In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
cle = pd.read_csv('../TrainTestData/cle_metadata.csv')
vir = pd.read_csv('../TrainTestData/vir_metadata.csv')
hun = pd.read_csv('../TrainTestData/hun_metadata.csv')
swi = pd.read_csv('../TrainTestData/swi_metadata.csv')

In [4]:
data = pd.concat([cle,vir,hun,swi])
data

,0,1,2,3,4,5,6,7,8,9,...,7.3,8.3,9.3,10.3,11.3,12.3,13.3,14.3,15.3,num
0,1.099659,0.0,0.0,0.872684,1.073222,0.0,1.486944,0.0,2.189892,0.0,...,0.0,1.457568,0.0,0.0,0.589020,0.882712,0.0,0.0,1.598649,0.00
1,1.849904,0.0,0.0,0.620106,0.811350,0.0,1.943316,0.0,2.594235,0.0,...,0.0,1.488225,0.0,0.0,0.637975,0.901482,0.0,0.0,1.643992,0.00
2,2.224025,0.0,0.0,0.119797,0.067982,0.0,2.657312,0.0,3.285271,0.0,...,0.0,1.199812,0.0,0.0,0.546282,0.756742,0.0,0.0,1.321362,0.00
3,1.738161,0.0,0.0,0.520788,0.593032,0.0,2.272287,0.0,2.677983,0.0,...,0.0,1.394613,0.0,0.0,0.627924,0.883689,0.0,0.0,1.631824,0.00
4,1.402882,0.0,0.0,0.570546,0.819004,0.0,1.494263,0.0,2.169592,0.0,...,0.0,1.280468,0.0,0.0,0.586185,0.768512,0.0,0.0,1.481180,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.352982,0.0,0.0,1.839385,2.316214,0.0,0.474900,0.0,1.077897,0.0,...,0.0,1.968976,0.0,0.0,0.851110,1.141455,0.0,0.0,2.026502,0.75
119,0.655624,0.0,0.0,1.390551,1.878792,0.0,0.452024,0.0,1.255075,0.0,...,0.0,1.707302,0.0,0.0,0.670807,1.002500,0.0,0.0,1.859282,0.50
120,0.161753,0.0,0.0,1.826201,2.245803,0.0,0.137622,0.0,0.954996,0.0,...,0.0,1.802060,0.0,0.0,0.740325,1.043203,0.0,0.0,1.887837,1.00
121,0.102779,0.0,0.0,1.974692,2.487005,0.0,0.091805,0.0,0.745667,0.0,...,0.0,1.693194,0.0,0.0,0.653663,1.047537,0.0,0.0,1.765075,0.25


In [5]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred))

Epoch 1/1000
20/20 [==============================] - 2s 63ms/step - loss: 0.4596 - accuracy: 0.7890
Epoch 2/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3629 - accuracy: 0.8571
Epoch 3/1000
20/20 [==============================] - 1s 64ms/step - loss: 0.3668 - accuracy: 0.8620
Epoch 4/1000
20/20 [==============================] - 1s 63ms/step - loss: 0.3539 - accuracy: 0.8555
Epoch 5/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3425 - accuracy: 0.8620
Epoch 6/1000
20/20 [==============================] - 1s 64ms/step - loss: 0.3583 - accuracy: 0.8636
Epoch 7/1000
20/20 [==============================] - 1s 61ms/step - loss: 0.3463 - accuracy: 0.8653
Epoch 8/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3332 - accuracy: 0.8701
Epoch 9/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3271 - accuracy: 0.8701
Epoch 10/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3332 - accura

In [7]:
model.save('../Models/Meta_Only/Meta_CNN.h5')

# Test on Each Data Set

In [12]:
def Test(path_train,path_test,model_name):
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]

    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = tf.keras.models.load_model(model_name)

    Y_pred = model.predict(X_test).argmax(axis=1)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred))

# Cleveland

In [15]:
path_train = '../TrainTestData/cle_metadata_train.csv'
path_test = '../TrainTestData/cle_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

3/3 [==============================] - 0s 12ms/step
[[41  3]
 [ 7 40]]
              precision    recall  f1-score   support

           0       0.93      0.85      0.89        48
           1       0.85      0.93      0.89        43

    accuracy                           0.89        91
   macro avg       0.89      0.89      0.89        91
weighted avg       0.89      0.89      0.89        91



# Virginia

In [16]:
path_train = '../TrainTestData/vir_metadata_train.csv'
path_test = '../TrainTestData/vir_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

2/2 [==============================] - 0s 14ms/step
[[ 8  2]
 [ 7 43]]
              precision    recall  f1-score   support

           0       0.80      0.53      0.64        15
           1       0.86      0.96      0.91        45

    accuracy                           0.85        60
   macro avg       0.83      0.74      0.77        60
weighted avg       0.85      0.85      0.84        60



# Hungarian

In [17]:
path_train = '../TrainTestData/hun_metadata_train.csv'
path_test = '../TrainTestData/hun_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

3/3 [==============================] - 0s 13ms/step
[[50  3]
 [ 5 31]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.93        55
           1       0.86      0.91      0.89        34

    accuracy                           0.91        89
   macro avg       0.90      0.91      0.91        89
weighted avg       0.91      0.91      0.91        89



# Switzerland

In [18]:
path_train = '../TrainTestData/swi_metadata_train.csv'
path_test = '../TrainTestData/swi_metadata_test.csv'
model = '../Models/Meta_Only/Meta_CNN.h5'
Test(path_train,path_test,model)

2/2 [==============================] - 0s 7ms/step
[[ 2  1]
 [ 1 33]]
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       0.97      0.97      0.97        34

    accuracy                           0.95        37
   macro avg       0.82      0.82      0.82        37
weighted avg       0.95      0.95      0.95        37

